# Task 3: Fine Tune NER Model 

In [2]:
# %pip install transformers datasets seqeval accelerate

In [1]:
from datasets import Dataset
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt 
import seaborn as sns   
sys.path.append(os.path.abspath("../scripts"))
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.chdir("..") 

In [3]:
def read_conll(path):
    sentences = []
    labels = []
    with open(path, "r", encoding="utf-8") as f:
        words = []
        tags = []
        for line in f:
            line = line.strip()
            if not line:
                if words:
                    sentences.append(words)
                    labels.append(tags)
                    words = []
                    tags = []
            else:
                splits = line.split()
                if len(splits) >= 2:
                    words.append(splits[0])
                    tags.append(splits[1])
        # Catch last sentence if no trailing newline
        if words:
            sentences.append(words)
            labels.append(tags)
    return pd.DataFrame({'tokens': sentences, 'ner_tags': labels})

df = read_conll("labeled_data.conll")
dataset = Dataset.from_pandas(df)

In [7]:
print(dataset[0])

{'tokens': ['ፎርኤቨር', 'ብራይት፦', 'ለጥርስዎ', 'ጥንካሬ', 'ፅዳት', 'እና', 'ንጣት', 'በአንድ', 'የያዘ', 'ከሬት', 'እና', 'ማር', 'የተቀመመ', 'የ', 'ምርት', 'ነው'], 'ner_tags': ['O', 'B-Product', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Product', 'O', 'B-Product', 'O', 'O', 'B-Product', 'O']}


In [ ]:
from transformers import AutoTokenizer

label_list = ['O', 'B-Product', 'I-Product', 'B-LOC', 'I-LOC', 'B-PRICE', 'I-PRICE']
label_to_id = {label: i for i, label in enumerate(label_list)}

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [1]:
def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(example["tokens"], is_split_into_words=True, truncation=True, padding="max_length", max_length=128)

    labels = []
    word_ids = tokenized_inputs.word_ids()
    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            labels.append(-100)
        elif word_idx != previous_word_idx:
            labels.append(label_to_id[example["ner_tags"][word_idx]])
        else:
            # Inside word piece: only keep I-... if applicable
            label = example["ner_tags"][word_idx]
            labels.append(label_to_id[label] if label.startswith("I-") else -100)
        previous_word_idx = word_idx

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [3]:
# tokenized_dataset = custom_dataset.map(tokenize_and_align_labels, batched=False)

In [ ]:
print(tokenized_dataset[0].keys())